*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
"""
Need to turn categorial data type into numerical numbers
"""
#The column title that are categorical
CATEGORICAL_COLUMNS = ['sex', 'smoker', 'region']

#For all the categorical column, keep them as categorical but use numeric 
#integers instead for each possible value.
for column in CATEGORICAL_COLUMNS:
  dataset[column] = dataset[column].astype('category').cat.codes


In [ ]:
"""
Split the data frain into 80% training and 20% testing. Then remove our desired
field that needs to be predicted (aka expenses) and put that into a variable.
"""
train_dataset, test_dataset = np.split(dataset, [int(.8 * len(dataset))])

#remove our expense column from our dataset, but put them into another variable
train_labels = train_dataset.pop('expenses')
test_labels = test_dataset.pop('expenses')



In [ ]:
"""
The dataset current isn't normalised and if the machine model reads it, it will
get confused since all the data have different reading method and decimal point
etc.
"""
#Create a normalizer object and the last axis input is a feature dimension
normalizer = layers.experimental.preprocessing.Normalization(axis = -1)

#Put our training data into the normalizer to normalise our data
normalizer.adapt(np.array(train_dataset))
normalizer

In [ ]:
"""
Create our model, set the optimizer, and compile it.
"""

#Initialise the model with the normalizer, and 2 layers of dense.
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=2),
    layers.Dense(units=1)

])

#Set the optimizer, and loss function. 
optim = keras.optimizers.Adam(learning_rate=0.2)

#compile our model
model.compile(
    optimizer = optim,
    loss = 'mae',
    metrics = ['mae', 'mse']
)

model.summary()

In [ ]:
"""
Train our model
"""

#Train the data with an epochs of 100
history = model.fit(
    train_dataset,
    train_labels,
    epochs = 100, 
    validation_split = 0.2,
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
